In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

In [106]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [0]:
transform = transforms.Compose([
                    transforms.Resize((32, 32)),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

root = "/content/drive/My Drive/Colab Notebooks/CS221/dataset"
full_set = datasets.ImageFolder(root=root, transform=transform)
train_size = int(0.8 * len(full_set))
test_size = len(full_set) - train_size
train_set, test_set = torch.utils.data.random_split(full_set, [train_size, test_size])


train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_set, shuffle=False, num_workers=0)


In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Flatten(torch.nn.Module):
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)

model = torch.nn.Sequential(
    nn.Linear(32 * 32 * 3, 1),
#     nn.Linear(32 * 32 * 3, 300),
#     nn.ReLU(),
    
#     nn.Linear(300, 300),
#     nn.LeakyReLU(),
    
#     nn.Linear(300, 1),
    nn.Sigmoid()
)

In [0]:
import torch.optim as optim

# Define the loss function and optimizer
loss_fn = torch.nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)


In [107]:
for epoch in range(10):
    # Train the model
    running_loss = 0.0
    model.train()
    for i, [inputs, labels] in enumerate(train_loader):   
        inputs = inputs.reshape(-1)
        outputs = model(inputs.float())
        loss = loss_fn(outputs, labels.float())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        running_loss += loss.item()
        
    # Test current version of model to obtain accuracy    
    correct = total = 0 
    with torch.no_grad():
        for [inputs, labels] in test_loader:   
            inputs = inputs.reshape(-1)
            probs = model(inputs.float())
            t = torch.Tensor([0.5])  # 0.5 acts as threshold
            outputs = (probs > t).float() * 1
            total += len(outputs)
            correct += (outputs == labels.float()).float().sum()
            
    accuracy = correct / total
    print ('Loss: {}     Accuracy: {}'.format(running_loss,  accuracy))


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 6. Skipping tag 41486
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41487
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Loss: 1007.2483580581902     Accuracy: 0.596875011920929
Loss: 933.2793135842294     Accuracy: 0.671875
Loss: 910.3854722561446     Accuracy: 0.628125011920929
Loss: 810.7282304397377     Accuracy: 0.668749988079071
Loss: 840.38466166126     Accuracy: 0.706250011920929
Loss: 845.4976892014201     Accuracy: 0.659375011920929
Loss: 801.0591343846718     Accuracy: 0.671875
Loss: 761.6414023453763     Accuracy: 0.640625
Loss: 695.5637461676779     Accuracy: 0.703125
Loss: 704.2515164338824     Accuracy: 0.6937500238418579
